In [1]:
#essential imports
import pandas as pd
import numpy as np

import datetime as dt
import time

# Strict JSON conversion
import json 
import gzip 
 
startyear=1990
startmonth=1

cps = 4
# how many month before the death of the artist are used as feature 
nbMonthBeforeDeath = 4

datapath = 'DATA/review_filtered/'
f_sbj_reviews = datapath+'movie_reviews_matched_'+str(cps)+'cps.csv'
f_ctl_reviews = datapath+'movie_reviews_matched_'+str(cps)+'cps_ctrls.csv'

# Load data

reviews related to subject (=dead) artists

In [2]:
sbj_reviews = pd.read_csv(f_sbj_reviews).drop('Unnamed: 0',axis=1)
sbj_reviews = sbj_reviews.rename(columns={'reviewTime' : 'strReviewTime', 'actor name' : 'artist name', 
                                          'musician name' : 'artist name', 'name' : 'artist name'})
sbj_reviews.head(3)

,index,asin,strReviewTime,overall,helpful,summary,reviewText,actor name,death date
0,308659,0792840461,2006-04-07,4.0,"[1, 3]",The orginal,The Original and the best-not much to say read...,jack weston,1996-05-03
1,308660,0792840461,2013-03-24,5.0,"[0, 0]",Steve McQueen invented cool,This was a great movie Steve McQueen invented ...,jack weston,1996-05-03
2,308661,0792840461,2014-04-05,4.0,"[1, 1]",Where The Dough Is,"Everybody knows banks, whether in storefronts,...",jack weston,1996-05-03


reviews related to control (=dead) artists

In [3]:
ctl_reviews = pd.read_csv(f_ctl_reviews).drop('Unnamed: 0',axis=1)
ctl_reviews = ctl_reviews.rename(columns={'reviewTime' : 'strReviewTime', 'actor name' : 'artist name', 
                                          'musician name' : 'artist name', 'name' : 'artist name'}})
ctl_reviews.head(3)

,index,asin,strReviewTime,overall,helpful,summary,reviewText,actor name
0,827321,630327417X,2009-10-15,5.0,"[0, 1]",Great Movie,Nothing bad to say. Dvd came quickly and in gr...,michael hynson
1,827322,630327417X,2000-09-16,5.0,"[0, 0]",endless summer,its just to good for words!,michael hynson
2,827323,630327417X,2005-03-03,1.0,"[5, 58]",A very disappointed lover of documentaries,I feel obligated to warn you-as I wish I was w...,michael hynson


# Pre-process data 

- transform string dates into formats that are easier for us to work with
- keep on the side the 'death date' <-> 'artist name relation 

Convert dates to year and extract and set aside actor<-> deathdates

In [5]:
# returns as an int the month of the date passed as argument. Apply to a serie !
def strdateToMonth(date) : 
    year = int(str(date)[:4])
    month = int(str(date)[5:7])
    assert(month <= 12 and month >= 1)
    return (month - startmonth) + (year - startyear)*12 

# returns as an int the year of the date passed as argument. Apply to a serie !
def strdateToYear(date) : 
    return int(str(date)[:4])

# SUBJECTS
sbj_reviews['reviewMonth'] = sbj_reviews['strReviewTime'].apply(strdateToMonth)
sbj_reviews['reviewYear'] = sbj_reviews['strReviewTime'].apply(strdateToYear)
sbj_reviews['reviewTime'] = pd.to_datetime(sbj_reviews['strReviewTime'])
sbj_reviews['death month'] = sbj_reviews['death date'].apply(strdateToMonth)
sbj_reviews['death year'] = sbj_reviews['death date'].apply(strdateToYear)
sbj_reviews['death date'] = pd.to_datetime(sbj_reviews['death date'])

# CONTROLS
ctl_reviews['reviewMonth'] = ctl_reviews['strReviewTime'].apply(strdateToMonth)
ctl_reviews['reviewYear'] = ctl_reviews['strReviewTime'].apply(strdateToYear)
ctl_reviews['reviewTime'] = pd.to_datetime(ctl_reviews['strReviewTime'])

Set aside the 'death date' <-> 'artist name relation 

In [6]:
deathdates = subjects_actors.groupby(['actor name','death date', 'death year', 'death month']).count()\
                            .reset_index()[['actor name', 'death date', 'death year', 'death month']]
deathdates.head()

,actor name,death date,death year,death month
0,adriana caselotti,1997-01-19,1997,84
1,adrienne shelly,2006-11-01,2006,202
2,al martino,2009-10-13,2009,237
3,alain bashung,2009-03-14,2009,230
4,alan bates,2003-12-27,2003,167


Extract controls names

In [7]:
#TODO : remove
#ctrl_names = list(set(controls_actors['actor name']))\
#sbj_names = list(deathdates['actor name'])

# filtering

### 'mentionned' Boolean

We want to distinguish which artists are frequently mentionned in their reviews; i.e, those that are popular, as opposed to those that just appear to play/sing with some other, more popular artist.
We add the boolean ['mentionned'] that tells if the artist listed with the review is mentionnned in the text of the review.

In [8]:
thresh_pop = 1

def mentionsName(row) : 
    if type(row['reviewText']) is str :
        names = row['actor name'].split(' ')
        text =  row['reviewText'].lower()
        # can mention only name or surname
        #return row['actor name'] in row['reviewText']
        for name in names : 
            if name in text : 
                return True
        return False
    else : return False

In [10]:
## Treat subjects
##
sbj_reviews['mentionned'] = sbj_reviews.apply(mentionsName, axis=1)
sbj_reviews.shape, sbj_reviews[sbj_reviews['mentionned']==True].shape

((597276, 15), (130234, 15))

In [12]:
## Treat controls
##
ctl_reviews['mentionned'] = ctl_reviews.apply(mentionsName, axis=1)
ctl_reviews.shape, ctl_reviews[controls_actors['mentionned']==True].shape

((285071, 12), (42178, 12))

Not necessarily important for later : the list of the most-mentionned subjects and controls

In [26]:
## Not necessarily needed, count of mentions are still in the reviews dataframe
##
sbj_popular = sbj_reviews[sbj_reviews['mentionned']==True]
sbj_popular = sbj_popular.groupby('actor name').count()['asin']
print(len(sbj_names), " reduced to", sbj_popular.shape)
sbj_popular.sort_values(ascending=False)[:5]

501  reduced to (460,)


actor name
james stewart      4135
paul newman        3119
charlton heston    3031
gregory peck       2618
marlon brando      2592
Name: asin, dtype: int64

In [13]:
## Not necessarily needed, count of mentions are still in the reviews dataframe
##
ctl_popular = ctl_reviews[ctl_reviews['mentionned']==True]
ctl_popular = ctl_popular.groupby('actor name').count()['asin']
print(len(ctrl_names), " reduced to", ctl_popular.shape)
ctl_popular.sort_values(ascending=False)[:5]

2419  reduced to (1215,)


actor name
matt damon            6017
steve martin          3266
tyler perry           2870
ian anderson          1213
laurence fishburne    1198
liev schreiber        1060
michelle williams     1058
marc m                 896
maggie gyllenhaal      867
robert taylor          749
Name: asin, dtype: int64

### Date filtering

We're only interested in the reviews of subjects that died in a recent time period because amazon's popularity is recent, so we'll have much more amazon data after year 2000. Therefore, we will only keep the reviews for which the subject died within a 2003-2014 time period, otherwise, the artist died too long ago.

TODO : RE-CHECK THIS, THIS WAS 1996-2013 ORIGINALLY
TODO : the filtered variable was bound to 'subjects in time' and not reused later on

In [14]:
start_year = 2003
end_year = 2014

print('original size : ', sbj_reviews.shape)
sbj_reviews = sbj_reviews[(sbj_reviews['death year']>=start) \
                        & (sbj_reviews['death year']<=end)]
print('filtered size : ', sbj_reviews.shape)
sbj_reviews.head(3)

original size :  (597276, 15)
filtered size :  (597276, 15)


,index,asin,strReviewTime,overall,helpful,summary,reviewText,actor name,death date,reviewMonth,reviewYear,reviewTime,death month,death year,mentionned
0,308659,0792840461,2006-04-07,4.0,"[1, 3]",The orginal,The Original and the best-not much to say read...,jack weston,1996-05-03,195,2006,2006-04-07,76,1996,False
1,308660,0792840461,2013-03-24,5.0,"[0, 0]",Steve McQueen invented cool,This was a great movie Steve McQueen invented ...,jack weston,1996-05-03,278,2013,2013-03-24,76,1996,False
2,308661,0792840461,2014-04-05,4.0,"[1, 1]",Where The Dough Is,"Everybody knows banks, whether in storefronts,...",jack weston,1996-05-03,291,2014,2014-04-05,76,1996,False


# Extract features used to match controls with subjects

We will use as features the number of review per month for a 4 months before the death of the subject. We obtain a features vector that we will try to match best between control and subjects.

Since the controls didn't die however, and since the amount of amazon reviews is generated increasingly fast as time passses, we will this feature for every control, for every subject we try to match it to, using the subject's death date as reference point and using the control's number of reviews one month before, two months before, and so on.

In [15]:
# relative count of months before death (-1, -2, ...)
relMonthBeforeDeath = [-i for i in range(1, nbMonthBeforeDeath+1)]
# string matching the feature's column in the subject's dataframe
relFeaturesStr = { i : 'revs'+str(i) for i in relMonthBeforeDeath}

In [16]:
# SUBJECTS COUNTS BY MONTH
sbj_reviews_monthly = sbj_reviews.groupby(['actor name','reviewMonth']).count()


''' returns the number of reviews for the subject in row, 'relMonthNb' months before his death month
'''
def getreview_month_subjects(row, relMonthNb) : 
    actor = row['actor name']
    deathMonth = int(row['death month'])
    try : 
        val = sbj_reviews_monthly.loc[actor, int(deathMonth+relMonthNb)]['asin']
        return int(val)
    ## Both exceptions can happen when the indexing keys are 
    ## not in the index == when there were no review for the actor at the given month
    except TypeError :
        #print('key (', actor, ', '+str(deathMonth+monthNb)+') returned typeError')
        return 0
    except KeyError :
        #print('key (', actor, ', ', deathMonth, ') found no match in the index')
        return 0
    

    
sbj_monthly_feats = pd.DataFrame(deathdates[['actor name', 'death month']])
for i in relMonthBeforeDeath :
    sbj_monthly_feats[relFeaturesStr[i]] = sbj_monthly_feats\
                                    .apply(lambda row : getreview_month_subjects(row, i), axis=1)

## Compute the norm of the feature
##
def feature_norm(row) : 
    feature = [row[relFeaturesStr[i]] for i in relMonthBeforeDeath]
    return np.linalg.norm(feature)

sbj_monthly_feats['featureNorm'] = sbj_monthly_feats.apply(feature_norm, axis=1)
sbj_monthly_feats['death date'] = deathdates['death date']
        
sbj_monthly_feats.sort_values(by='featureNorm', ascending=False).head()

,actor name,death month,revs-1,revs-2,revs-3,revs-4,featureNorm,death date
357,mickey rooney,291,149,156,236,181,367.415296,2014-04-06
228,james garner,294,151,178,169,196,348.513988,2014-07-19
140,eli wallach,293,120,138,114,148,261.426854,2014-06-24
389,philip seymour hoffman,289,119,106,68,77,189.604852,2014-02-02
288,karen black,283,74,111,121,39,184.279679,2013-08-08


# Matching

Prepare the control's dataframe for matching, and create a function that computes the features for subjects

In [ ]:
ctl_reviews_sorted = ctl_reviews.groupby(['actor name','reviewMonth'])\
                                .count()['asin']\
                                .to_frame().reset_index()
        
ctl_reviews_sorted.sort_values(by=['actor name', 'asin'], ascending=False).head()        

In [20]:
''' returns the number of reviews for the control in row, 
    looking in 'musiciansDataframe', 'relMonthNb' months before 'deathMonth'
'''
def getreview_month_controls(row, monthNb, musiciansDataframe, deathMonth) : 
    actor = row['actor name']
    try : 
        val = musiciansDataframe.loc[actor, int(deathMonth+monthNb)]['asin']
        return int(val)
    ## Both exceptions can happen when the indexing keys are 
    ## not in the index == when there were no review for the actor at the given month
    except TypeError :
        #print('key (', actor, ', '+str(deathMonth+monthNb)+') returned typeError')
        return 0
    except KeyError :
        #print('key (', actor, ', ', deathMonth, ') found no match in the index')
        return 0

In [21]:
subjects_monthly_feats.head()
controls_revs.set_index(['actor name','reviewMonth']).loc['adrian holmes']
controls_revs.groupby('actor name').count().reset_index().head()

,actor name,reviewMonth,asin
0,aaron chayrez,1,1
1,aaron kwok,60,60
2,aarti chopra,2,2
3,abdul salaam el razzac,5,5
4,adam clark,19,19


### Matching algorithm : 
   
   
Match is a greedy function, that given a threshold and a subject, will iterate over the controls, compare the features, and if the difference is below a given threshold, call it a match. If the threshold cannot be met, it returns that it couldn't match.
We call this function over all the subjects, increasing the threshold until a match can be found. 

In [22]:
# helper to obtain a value after indexing on a dataframe
def unwrap(subjectStuff) : 
    return subjectStuff.tolist()[0]

def match(artistname, threshold, ctrl_revs) : 

    # extract the useful features for the subject 
    subject = sbj_monthly_feats[sbj_monthly_feats['artist name'] == artistname]
    deathmonth = int(subject['death month'])
    sbjname = unwrap(subject['artist name'])
    feature = [unwrap(subject[relFeaturesStr[i]]) for i in relMonthBeforeDeath]
    
    #print(sbj_actor)
    #print('feature : ', feature)
    
    for control in ctrl_revs.groupby('artist name').count().reset_index().iterrows() : 
        
        # iterrows returns tuple (0, row)
        idx = control[0]
        control = control[1]
        
        # Compute feature
        ctrl_feature = [getreview_month_controls(control, i, 
                                        ctrl_revs.set_index(['artist name', 'reviewMonth']), 
                                        deathMonth=deathmonth) \
                        for i in relMonthBeforeDeath]
        
        #print('   ctrl_feature : ', ctrl_feature )
        
        diff = np.linalg.norm(np.array(feature) - np.array(ctrl_feature))
        
        #print('    diff : ', diff)
        
        if diff <= threshold : 
            matched_name = control['artist name']
            matched_count = ctrl_feature
            
            #ctrl_revs = ctrl_revs[ctrl_revs['actor name'].isin([matched_name])==False]
            ctrl_revs = ctrl_revs[ctrl_revs['artist name']!=matched_name]
            
            print('    ', sbjname, ' matched with ', matched_name)
            print('    ', feature, ' ~ ' , ctrl_feature)
            return matched_name, ctrl_feature, ctrl_revs
    
    print('1 iteration completed over all controls')
    return 0, 0, ctrl_revs  


In [25]:
ctrl_revs = ctl_reviews_sorted.copy()
sbj_monthly_feats_sorted = sbj_monthly_feats.sort_values(by='featureNorm', ascending=False)

sbj_list = []
sbj_feat_list = []
sbj_death_list = []
ctl_list = []
ctl_feat_list = []

for idx, subject_name in enumerate(sbj_monthly_feats_sorted['artist name']):
    
    if idx%50 == 0: 
        print(idx, " subjects out of ", len(sbj_monthly_feats['artist name']))
    
    # extract the feature for the subject and use it to set the threshold
    subject = sbj_monthly_feats_sorted[sbj_monthly_feats_sorted['artist name'] == subject_name]
    feature = [unwrap(subject[relFeaturesStr[i]]) for i in relMonthBeforeDeath]
    
    sbjname = unwrap(subject['artist name'])
    print(sbjname, '-', feature)
    
    featureNorm = subject['featureNorm']
    thr = int(8 + 0.12*featureNorm)
    mtch = 0

    while mtch == 0:
        
        if thr > 50 and thr < 60  : 
            print(sbjname+' reached threshold ', thr)
        if thr > 1000 and thr < 1250  : 
            print(sbjname+' reached threshold ', thr)
        
        name, ctrl_feature, ctrl_revs_ = match(sbjname,thr,ctrl_revs)
        ctrl_revs = ctrl_revs_.sample(frac=1) 
        
        
        if name != 0:
            
            #print(subject_name+' matched with '+name)
            
            sbjDeathDate = unwrap(sbj_actor['death date'])
            
            sbj_list.append(sbjname)
            sbj_feat_list.append(feature)
            sbj_death_list.append(sbjDeathDate)
            ctl_list.append(name)
            ctl_feat_list.append(ctrl_feature)
            
            mtch += 1
            
        
        thr += int(0.3*thr + 1)
        print('  thr increased: ', thr)
            
print('DONE')

2419 501
0  subjects out of  500
mickey rooney - [149, 156, 236, 181]
  thr :  79
     mickey rooney  matched with  liev schreiber
     [149, 156, 236, 181]  ~  [158, 201, 257, 241]
james garner - [151, 178, 169, 196]
  thr :  74
     james garner  matched with  laurence fishburne
     [151, 178, 169, 196]  ~  [144, 155, 144, 208]
eli wallach - [120, 138, 114, 148]
  thr :  59
eli wallach reached threshold  59
     eli wallach  matched with  ben miles
     [120, 138, 114, 148]  ~  [94, 102, 120, 138]
philip seymour hoffman - [119, 106, 68, 77]
  thr :  46
     philip seymour hoffman  matched with  cheech marin
     [119, 106, 68, 77]  ~  [100, 95, 43, 54]
karen black - [74, 111, 121, 39]
  thr :  46
1 iteration completed over all controls
  thr :  70
     karen black  matched with  albert brooks
     [74, 111, 121, 39]  ~  [91, 98, 72, 76]
bob hoskins - [97, 88, 95, 87]
  thr :  46
     bob hoskins  matched with  maggie gyllenhaal
     [97, 88, 95, 87]  ~  [90, 71, 90, 50]
julie harris

     heath ledger  matched with  irene dunne
     [18, 12, 11, 15]  ~  [14, 3, 4, 4]
elizabeth taylor - [10, 21, 11, 11]
  thr :  17
     elizabeth taylor  matched with  elise gulan
     [10, 21, 11, 11]  ~  [7, 15, 7, 8]
deborah kerr - [7, 10, 15, 20]
  thr :  17
     deborah kerr  matched with  jason douglas
     [7, 10, 15, 20]  ~  [5, 8, 7, 17]
gregory peck - [13, 7, 14, 18]
  thr :  17
     gregory peck  matched with  kip pardue
     [13, 7, 14, 18]  ~  [7, 9, 11, 12]
andy griffith - [13, 14, 10, 16]
  thr :  17
     andy griffith  matched with  jean marsh
     [13, 14, 10, 16]  ~  [4, 6, 7, 12]
victor wong - [11, 7, 17, 16]
  thr :  17
     victor wong  matched with  bill capizzi
     [11, 7, 17, 16]  ~  [12, 14, 22, 2]
jack lemmon - [13, 8, 7, 19]
  thr :  17
     jack lemmon  matched with  masaya onosaka
     [13, 8, 7, 19]  ~  [4, 12, 8, 8]
mary kay bergman - [25, 4, 0, 0]
  thr :  17
     mary kay bergman  matched with  michael dorn
     [25, 4, 0, 0]  ~  [11, 2, 2, 1]
larry 

     robert culp  matched with  ajay chaudhary
     [7, 9, 1, 4]  ~  [0, 0, 0, 1]
john mills - [9, 4, 6, 3]
  thr :  14
     john mills  matched with  alan dowell
     [9, 4, 6, 3]  ~  [0, 0, 0, 0]
deforest kelley - [6, 5, 5, 7]
  thr :  14
     deforest kelley  matched with  alan parry
     [6, 5, 5, 7]  ~  [0, 0, 0, 0]
bernadette lafont - [5, 8, 3, 6]
  thr :  14
     bernadette lafont  matched with  alan stemm
     [5, 8, 3, 6]  ~  [0, 0, 0, 0]
nigel hawthorne - [6, 5, 6, 6]
  thr :  14
     nigel hawthorne  matched with  alberta mayberry
     [6, 5, 6, 6]  ~  [0, 0, 0, 0]
peter ustinov - [2, 8, 4, 7]
  thr :  14
     peter ustinov  matched with  alberto alejandrino
     [2, 8, 4, 7]  ~  [0, 0, 0, 0]
kevin mccarthy - [10, 2, 4, 3]
  thr :  14
     kevin mccarthy  matched with  alberto barros
     [10, 2, 4, 3]  ~  [0, 0, 0, 0]
betty hutton - [5, 10, 1, 0]
  thr :  14
     betty hutton  matched with  albie casino
     [5, 10, 1, 0]  ~  [0, 0, 0, 0]
evelyn keyes - [2, 7, 6, 6]
  thr :

     eartha kitt  matched with  angela rovetto nora salinas
     [3, 3, 2, 2]  ~  [0, 0, 0, 0]
tomoko kawakami - [5, 1, 0, 0]
  thr :  13
     tomoko kawakami  matched with  angela schijf
     [5, 1, 0, 0]  ~  [0, 0, 0, 0]
oliver reed - [1, 2, 2, 4]
  thr :  13
     oliver reed  matched with  angeles cruz
     [1, 2, 2, 4]  ~  [0, 0, 0, 0]
leslie cheung - [0, 2, 4, 2]
  thr :  13
     leslie cheung  matched with  angelica kirchshlager
     [0, 2, 4, 2]  ~  [0, 0, 0, 0]
mel ferrer - [4, 2, 0, 2]
  thr :  13
     mel ferrer  matched with  angelina estrada
     [4, 2, 0, 2]  ~  [0, 0, 0, 0]
jill clayburgh - [3, 3, 2, 0]
  thr :  13
     jill clayburgh  matched with  angelina pelaez
     [3, 3, 2, 0]  ~  [0, 0, 0, 0]
sylvia kristel - [2, 1, 4, 1]
  thr :  13
     sylvia kristel  matched with  angeline wynn
     [2, 1, 4, 1]  ~  [0, 0, 0, 0]
gloria stuart - [1, 4, 0, 2]
  thr :  13
     gloria stuart  matched with  angelo russo
     [1, 4, 0, 2]  ~  [0, 0, 0, 0]
michael jeter - [0, 4, 2, 1]

     matthew mcgrory  matched with  bacarian radio choir
     [1, 0, 0, 1]  ~  [0, 0, 0, 0]
dolphy - [0, 1, 1, 0]
  thr :  13
     dolphy  matched with  bailey handler & aztec
     [0, 1, 1, 0]  ~  [0, 0, 0, 0]
troy donahue - [0, 1, 0, 1]
  thr :  13
     troy donahue  matched with  bailey munoz
     [0, 1, 0, 1]  ~  [0, 0, 0, 0]
marie trintignant - [0, 1, 1, 0]
  thr :  13
     marie trintignant  matched with  barbara eda-young
     [0, 1, 1, 0]  ~  [0, 1, 0, 1]
joan sims - [1, 0, 1, 0]
  thr :  13
     joan sims  matched with  barbara hall
     [1, 0, 1, 0]  ~  [0, 0, 0, 0]
ray harryhausen - [0, 1, 1, 0]
  thr :  13
     ray harryhausen  matched with  barclay hope
     [0, 1, 1, 0]  ~  [1, 4, 0, 0]
anita page - [1, 0, 0, 1]
  thr :  13
     anita page  matched with  barret helms
     [1, 0, 0, 1]  ~  [0, 0, 0, 0]
al martino - [0, 0, 1, 0]
  thr :  13
     al martino  matched with  barry a ritzler
     [0, 0, 1, 0]  ~  [0, 0, 0, 0]
mabel king - [0, 0, 0, 1]
  thr :  13
     mabel king

     theo van gogh  matched with  bob swerer sr
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
shammi kapoor - [0, 0, 0, 0]
  thr :  13
     shammi kapoor  matched with  bobby ellis
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
theresa merritt - [0, 0, 0, 0]
  thr :  13
     theresa merritt  matched with  bobby sox
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
phil hartman - [0, 0, 0, 0]
  thr :  13
     phil hartman  matched with  bohuslav srámek
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
sidney lumet - [0, 0, 0, 0]
  thr :  13
     sidney lumet  matched with  bonnie rideout
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
siegfried lowitz - [0, 0, 0, 0]
  thr :  13
     siegfried lowitz  matched with  bonnie soaper
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
poul bundgaard - [0, 0, 0, 0]
  thr :  13
     poul bundgaard  matched with  brad moranz
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
sergiu nicolaescu - [0, 0, 0, 0]
  thr :  13
     sergiu nicolaescu  matched with  brandon davison-tracy
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
sivaji ganesan - [0, 0, 0, 0]


     audra lindley  matched with  catherine belkhodja
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
arik einstein - [0, 0, 0, 0]
  thr :  13
     arik einstein  matched with  catherine byrne
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
arihiro hase - [0, 0, 0, 0]
  thr :  13
     arihiro hase  matched with  catherine fitch
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
arch johnson - [0, 0, 0, 0]
  thr :  13
     arch johnson  matched with  catherine hickland
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
anne haddy - [0, 0, 0, 0]
  thr :  13
     anne haddy  matched with  cathy kinney
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
400  subjects out of  500
annabella - [0, 0, 0, 0]
  thr :  13
     annabella  matched with  ceci moses
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
amália rodrigues - [0, 0, 0, 0]
  thr :  13
     amália rodrigues  matched with  cecily fay
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
alvy moore - [0, 0, 0, 0]
  thr :  13
     alvy moore  matched with  celeste collins
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
aliki vougiouklaki - [0, 0, 0, 

     lolo ferrari  matched with  claire borotra
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
lois maxwell - [0, 0, 0, 0]
  thr :  13
     lois maxwell  matched with  claire de lorez
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
jorge russek - [0, 0, 0, 0]
  thr :  13
     jorge russek  matched with  clark smith
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
jon pertwee - [0, 0, 0, 0]
  thr :  13
     jon pertwee  matched with  claudia abbate
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
john hughes - [0, 0, 0, 0]
  thr :  13
     john hughes  matched with  claudine wilde
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
jack nance - [0, 0, 0, 0]
  thr :  13
     jack nance  matched with  claudio garcia satur
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
jack lord - [0, 0, 0, 0]
  thr :  13
     jack lord  matched with  clay marzo
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
jack cardiff - [0, 0, 0, 0]
  thr :  13
     jack cardiff  matched with  clayton taylor
     [0, 0, 0, 0]  ~  [0, 0, 0, 0]
irene hervey - [0, 0, 0, 0]
  thr :  13
     irene hervey  matched wit

# Results

In [27]:
res = pd.DataFrame()

res['sbj'] = sbj_list
res['sbj_feats'] = sbj_feat_list
res['death date'] = sbj_death_list
res['ctl_feats'] = ctl_feat_list
res['ctl'] = ctl_list

res.head()

,sbj,sbj_feats,death date,ctl_feats,ctl
0,mickey rooney,"[149, 156, 236, 181]",2014-04-06,"[158, 201, 257, 241]",liev schreiber
1,james garner,"[151, 178, 169, 196]",2014-07-19,"[144, 155, 144, 208]",laurence fishburne
2,eli wallach,"[120, 138, 114, 148]",2014-06-24,"[94, 102, 120, 138]",ben miles
3,philip seymour hoffman,"[119, 106, 68, 77]",2014-02-02,"[100, 95, 43, 54]",cheech marin
4,karen black,"[74, 111, 121, 39]",2013-08-08,"[91, 98, 72, 76]",albert brooks


In [ ]:
# TODO - REMOVE
'''
ctrl_features = pd.DataFrame(ctrl_list, columns=['ctrl_name'])
ctrl_features['ctrl_reviews'] = count_list

res = pd.concat([ctrl_features.reset_index(), sorted_subjects.reset_index()], axis=1)\
            .drop('index', axis=1)
res['error'] = np.abs(res['ctrl_reviews'] - res['reviews'])/(2*(1+res['ctrl_reviews'] + res['reviews']))

res.sort_values(by='error', ascending=False).head()
'''

In [ ]:
# TODO - REMOVE
#ctrl_features.hist(column='ctrl_reviews', figsize=[10,10], bins=100)

In [ ]:
# TODO - REMOVE
#subjects_feats.hist(column='reviews', figsize=[10,10], bins=100)

# Export results

In [31]:
subjects_nb = str(res.shape[0]-1)

In [32]:
subjects_actors.to_csv('reviews_subjects_'+subjectsnb+'_vectormatched.csv', index=False)
controls_actors.to_csv('reviews_controls_'+subjectsnb+'_vectormatched.csv', index=False)